In [1]:
# Importação das bibliotecas necessárias
import cv2
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow # Para exibir imagens

In [2]:
# Funções Principais

def montar_sistema_poisson(img, mascara):
  """
  Monta o sistema linear esparso Ax=b baseado na equação de Poisson.
  A equação é: 4*u(i,j) - u(i-1,j) - u(i+1,j) - u(i,j-1) - u(i,j+1) = b_val
  """

  altura, largura = img.shape

  # Identificando incógnitas (pixels ausentes/danificados)
  # np.where(mascara) retorna uma tupla (array_linhas, array_colunas)
  pixels_ausentes = np.where(mascara)

  # N é o número total de pixels
  N = len(pixels_ausentes[0])

  # Mapear coordenadas (i, j) para o índice k (0 a N-1) do sistema linear
  mapa_pixels = np.full(img.shape, -1, dtype=int)
  mapa_pixels[pixels_ausentes] = np.arange(N)

  # Montar a matriz A (N x N) e vetor b (N x 1)
  A = sp.lil_matrix((N, N))
  b = np.zeros(N)

  for k in range(N):
        i, j = pixels_ausentes[0][k], pixels_ausentes[1][k]

        # Define a diagonal principal
        A[k, k] = 4

        # Analisa os 4 vizinhos
        for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            viz_i, viz_j = i + di, j + dj

            if mascara[viz_i, viz_j]:
                viz_k = mapa_pixels[viz_i, viz_j] # Encontra o índice 'm' do vizinho
                A[k, viz_k] = -1 # Define A[k, m] = -1

            else:
                b[k] += img[viz_i, viz_j]

  # Converte A para o formato CSR (Compressed Sparse Row) para um cálculo rápido
  A_csr = A.tocsr()

  return A_csr, b, mapa_pixels